In [1]:
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceMatrix, DistanceTreeConstructor
from keyname import keyname as kn
from matplotlib import colors as mpl_colors
from teeplot import teeplot as tp

import Bio
import alifedata_phyloinformatics_convert as apc
import csv
import glob
import itertools
import json
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import networkx as nx
import numpy as np
import pandas as pd
import scipy as sci
import seaborn as sns
import urllib.request as request
import io
from copy import deepcopy


from etlib.SortableTree.SortableTree import SortableTree, find_leaves, to_tril
from etlib.PlotTools.PlotTools import enable_pretty_graphing
enable_pretty_graphing()

In [3]:
morphs = {x['stint']: x[' morph'][1:] for x in pd.read_csv('morphs_by_stint.csv').to_dict(orient='records')}
matrix = np.loadtxt("data/distance_matrix.matrix")
with open("data/stint_tags.json", 'r') as f:
    stint_tags = json.loads(f.read())

In [6]:
scorer = Phylo.TreeConstruction.ParsimonyScorer()

In [7]:
searcher = Phylo.TreeConstruction.NNITreeSearcher(scorer)

In [8]:
dm = Bio.Phylo.TreeConstruction.DistanceMatrix(
    names=[str(x) for x in range(len(stint_tags))],
    matrix=to_tril(matrix.T)
)
tree = getattr(DistanceTreeConstructor(), 'nj')(dm)

In [9]:
bigtags_per_stint = {}

for stint, tags in stint_tags.items():
    bigtag = ''.join(bin(x)[2:].zfill(64) for x in tags)
    bigtags_per_stint[stint] = bigtag

In [10]:
align_str = f"""{len(bigtags_per_stint)} {len(bigtags_per_stint[0])}
"""

for stint, bigtag in bigtags_per_stint.items():
    align_str += str(stint).zfill(10) + str(bigtag) + "\n"

In [11]:
f = io.StringIO(align_str)

In [12]:
from Bio import AlignIO
aln = AlignIO.read(f, 'phylip')
aln

<<class 'Bio.Align.MultipleSeqAlignment'> instance (101 records of length 2240) at 7f36ab6c21d0>

In [13]:
def rename_tree(root):
    try:
        stint_id = int(root.name)
        root.name = str(stint_id).zfill(10)
    except (ValueError, TypeError):
        pass
    for node in root.clades:
        rename_tree(node)

In [14]:
fixed_tree = deepcopy(tree)
rename_tree(fixed_tree.clade)

In [15]:
constructor = Phylo.TreeConstruction.ParsimonyTreeConstructor(searcher, fixed_tree)

In [16]:
pars_tree = constructor.build_tree(aln)

In [1]:
sorted_pars_tree = deepcopy(pars_tree)
sorted_pars_tree = SortableTree(sorted_pars_tree.clade)
sorted_pars_tree.sort_tree(reverse=True)

NameError: name 'deepcopy' is not defined

In [20]:
rerooted_pars_tree = deepcopy(sorted_pars_tree)
rerooted_pars_tree.root_with_outgroup({"name": "0000000002"})
rerooted_pars_tree.sort_tree(reverse=True)

In [22]:
# store parsimony trees
Phylo.write(pars_tree, "data/parsimony_tree.xml", "phyloxml")
Phylo.write(sorted_pars_tree, "data/sorted_parsimony_tree.xml", "phyloxml")
Phylo.write(rerooted_pars_tree, "data/rerooted_pars_tree.xml", "phyloxml")

1